In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, f1_score, accuracy_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, auc, roc_curve
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from statsmodels.stats.outliers_influence import variance_inflation_factor  
import warnings
import pickle
import os
import json


warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [2]:
cols = ['ts', 'uid', 'id.orig_h', 'id.orig_p',
        'id.resp_h', 'id.resp_p', 'proto', 'service',
        'duration',  'orig_bytes', 'resp_bytes',
        'conn_state', 'local_orig', 'local_resp',
        'missed_bytes',  'history', 'orig_pkts',
        'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes',
        'tunnel_parents', 'label']

In [3]:
out_dir = f'/home/meryem.janati/lustre/nlp_team-um6p-st-sccs-id7fz1zvotk/IDS/janati/IDS/Datasets/unsw/Zeek/17-02-2015/timeout60'
df1 = pd.read_csv(out_dir+"/UNSW-NB15_zeek_60.csv")

In [4]:
df1.columns

Index(['ts', 'uid', 'id.orig_h', 'id.orig_p', 'id.resp_h', 'id.resp_p',
       'proto', 'service', 'duration', 'orig_bytes', 'resp_bytes',
       'conn_state', 'local_orig', 'local_resp', 'missed_bytes', 'history',
       'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes',
       'tunnel_parents', 'id', 'Attack'],
      dtype='object')

In [31]:
out_dir = f'/home/meryem.janati/lustre/nlp_team-um6p-st-sccs-id7fz1zvotk/IDS/janati/IDS/Datasets/unsw/Zeek/22-01-2015/timeout60'
df2 = pd.read_csv(out_dir+"/UNSW-NB15_zeek_60.csv")
df2.shape

(1028283, 23)

In [32]:
df = pd.concat([df1, df2], ignore_index=True, sort=False)

In [33]:
df.shape

(2056606, 23)

In [34]:
df

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents,id,Attack
0,1.424223e+09,C9dAU041N3HqL5K6kd,59.166.0.7,61953,149.171.126.2,24075,tcp,-,0.05984,236,0,S0,F,F,0,Sa,2,112,2,104,-,0,Benign
1,1.424223e+09,CJ1IZu1BweGN34fIia,149.171.126.5,6881,59.166.0.4,65360,tcp,-,0.122888,64154,68,SF,F,F,0,DTaTdtAFf,68,81674,37,2009,-,1,Benign
2,1.424223e+09,CCur3k3zLi0KpW626,59.166.0.9,24935,149.171.126.2,5190,tcp,-,0.004947,216,814,SF,F,F,0,ShADTdtfFa,12,1064,12,2260,-,2,Benign
3,1.424223e+09,CWTE0433ZFQyGfmTzl,59.166.0.6,37415,149.171.126.1,143,tcp,-,0.137649,68,225,SF,F,F,0,DTadtAfF,8,502,14,1094,-,3,Benign
4,1.424223e+09,CZylbB1X08yVSPfALe,59.166.0.7,25718,149.171.126.2,23652,tcp,-,0.074317,235,22843,SF,F,F,3967,ShdGtAgtfFa,32,1672,35,40046,-,4,Benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2056601,1.421932e+09,CnYWe54BO49XHsW88h,175.45.176.0,1034,149.171.126.11,520,udp,-,0.000012,1048,0,S0,F,F,0,D,2,1104,0,0,-,1028277,Fuzzers
2056602,1.421928e+09,C9ZdaR3kpQ1BmrbpU7,149.171.126.15,179,175.45.176.3,41558,tcp,-,0.074259,0,0,SF,F,F,0,AfFa,6,240,4,160,-,227010,direction_flip:Fuzzers
2056603,1.421928e+09,COGrCkrzqYLs9qGj7,149.171.126.16,1027,175.45.176.0,42174,tcp,dce_rpc,0.244165,60,6848,SF,F,F,0,DTdtFfaA,6,360,14,14256,-,227012,direction_flip:Exploits
2056604,1.421931e+09,C4kLLs41TF8OZnO1K8,149.171.126.15,89,175.45.176.0,28433,tcp,-,0.021418,0,0,SF,F,F,0,FfaA,4,160,4,160,-,343402,direction_flip:Exploits


In [5]:
def save_scores(timeout, meanScores, stdScore):
    results = {
        'Timeout': timeout,
        'Mean of all scores': meanScores,
        'Std of all Scores': stdScores

    }

    with open(f'../Checkpoints/ET/ET_unsw_zeek_{timeout}.json', 'w') as f:
        json.dump(results, f, indent=4)

In [27]:
timeouts = ['default', 0.5, 1, 2, 3, 4, 5, 6, 10, 30, 60]

# Training

In [28]:
best_f1 = 0
worst_f1 = 1
best_mean, worst_mean, best_std, worst_std = None, None, None, None

save=True

for timeout in timeouts:
    print("Processing timeout : ", timeout)
    if timeout =='default':
        out_dir1 = f'/home/meryem.janati/lustre/nlp_team-um6p-st-sccs-id7fz1zvotk/IDS/janati/IDS/Datasets/unsw/Zeek/17-02-2015/{timeout}/UNSW-NB15_zeek_{timeout}.csv'
        out_dir2 = f'/home/meryem.janati/lustre/nlp_team-um6p-st-sccs-id7fz1zvotk/IDS/janati/IDS/Datasets/unsw/Zeek/22-01-2015/{timeout}/UNSW-NB15_zeek_{timeout}.csv'
    else:
        out_dir1 = f'/home/meryem.janati/lustre/nlp_team-um6p-st-sccs-id7fz1zvotk/IDS/janati/IDS/Datasets/unsw/Zeek/17-02-2015/timeout{timeout}/UNSW-NB15_zeek_{timeout}.csv'
        out_dir2 = f'/home/meryem.janati/lustre/nlp_team-um6p-st-sccs-id7fz1zvotk/IDS/janati/IDS/Datasets/unsw/Zeek/22-01-2015/timeout{timeout}/UNSW-NB15_zeek_{timeout}.csv'

    df1 = pd.read_csv(out_dir1)
    df2 = pd.read_csv(out_dir2)
    
    df = pd.concat([df1, df2], ignore_index=True, sort=False)
    
    
    df = df.drop(columns=['id', 'uid', 'id.orig_h', 'id.resp_h', 'tunnel_parents']) # tunnel_parents is empty
    #df = df[~df['Attack'].str.startswith('direction_flip')]


    # Handle missing values (if any)
    df.replace({'orig_bytes': '-'}, '0', inplace=True)
    df['orig_bytes'] = pd.to_numeric(df['orig_bytes'], errors='coerce')
    df['orig_bytes'] = df['orig_bytes'].fillna(0).astype('int64')

    df.replace({'resp_bytes': '-'}, '0', inplace=True)
    df['resp_bytes'] = pd.to_numeric(df['resp_bytes'], errors='coerce')
    df['resp_bytes'] = df['resp_bytes'].fillna(0).astype('int64')


    df.replace({'duration': '-'}, '0', inplace=True)
    df['duration'] = pd.to_numeric(df['duration'], errors='coerce')
    df['duration'] = df['duration'].fillna(0).astype('float64')

    df['service'] = df['service'].replace('-', np.nan)
    df['history'] = df['history'].replace('-', np.nan)
    
    # Convert categorical variables to numerical using Label Encoding
    # Encode protocol and service types
    label_encoders = {}
    for column in ['proto', 'service', 'conn_state', 'history', 'local_orig', 'local_resp']:
        if column in df.columns:
            le = LabelEncoder()
            df[column] = le.fit_transform(df[column])
            label_encoders[column] = le

    # Split df into features and labels
    X = df.drop(columns=['Attack'])  # Assuming 'label' is the target variable
    y = df['Attack']
    
    accuracy, f1, precision, recall =[], [], [], []
    
    skf= StratifiedKFold(n_splits=5,random_state=None)
    skf.get_n_splits(X,y)
    
    for (train_index, test_index), i in zip(skf.split(X, y), range(5)):
        X_train,X_test=X.iloc[train_index],X.iloc[test_index]
        y_train,y_test=y.iloc[train_index],y.iloc[test_index]


        le = LabelEncoder()
        y_train = le.fit_transform(y_train)
        
        
        # Map unseen values to '<unknown>'
        y_test = y_test.map(lambda s: '<unknown>' if s not in le.classes_ else s)

        # Add '<unknown>' to classes and transform
        le.classes_ = np.append(le.classes_, '<unknown>')
        y_test = le.transform(y_test)
        
        
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        # Initialize and train Extra Trees Classifier
        clf = ExtraTreesClassifier(n_estimators=100, random_state=42) #RandomForestClassifier(random_state=42, class_weight='balanced') 
        clf.fit(X_train, y_train)
        pred = clf.predict(X_test)
        
        f1Score = f1_score(y_true=y_test, y_pred=pred, average='macro')
        accScore=accuracy_score(y_test, pred)
        precScore = precision_score(y_test, pred, average='macro')
        recScrore = recall_score(y_test, pred, average='macro')
                             
        f1.append(f1Score)
        accuracy.append(accScore)
        precision.append(precScore)
        recall.append(recScrore)
        print('Fold: ', i, 'done!')

    meanScores, stdScores = {}, {}
    
    meanScores['f1Mean'] = np.array(f1).mean()
    meanScores['accMean'] = np.array(accuracy).mean()
    meanScores['recMean'] = np.array(recall).mean()
    meanScores['precMean'] = np.array(precision).mean()
    
    stdScores['f1Std'] = np.array(f1).std()
    stdScores['accStd'] = np.array(accuracy).std()
    stdScores['recStd'] = np.array(recall).std()
    stdScores['precStd'] = np.array(precision).std()
    
    print("Mean of all scores: ", meanScores)
    print("Std of all scores: ", stdScores)


    if save:
        save_scores(timeout, meanScores, stdScores)

    if meanScores['f1Mean'] > best_f1: 
        best_timeout = timeout
        best_mean = meanScores
        best_std = stdScores
        best_f1 = meanScores['f1Mean']
    
    if meanScores['f1Mean'] <= worst_f1: 
        
        worst_timeout = timeout
        worst_mean = meanScores
        worst_std = stdScores
        worst_f1 = meanScores['f1Mean']
               
    print('_______________________________________________')

Processing timeout :  default
Fold:  0 done!
Fold:  1 done!
Fold:  2 done!
Fold:  3 done!
Fold:  4 done!
Mean of all scores:  {'f1Mean': 0.43021937379884506, 'accMean': 0.976267093586246, 'recMean': 0.40952987461691004, 'precMean': 0.5597863118334097}
Std of all scores:  {'f1Std': 0.09988257723838624, 'accStd': 0.005716066239009134, 'recStd': 0.12048625261275438, 'precStd': 0.08796330117133426}
_______________________________________________
Processing timeout :  0.5
Fold:  0 done!
Fold:  1 done!
Fold:  2 done!
Fold:  3 done!
Fold:  4 done!
Mean of all scores:  {'f1Mean': 0.3962000542518317, 'accMean': 0.9763312766246365, 'recMean': 0.38832241291414926, 'precMean': 0.5183943543788683}
Std of all scores:  {'f1Std': 0.08723040793322173, 'accStd': 0.005675118223453088, 'recStd': 0.11082268873921883, 'precStd': 0.09006772797547785}
_______________________________________________
Processing timeout :  1
Fold:  0 done!
Fold:  1 done!
Fold:  2 done!
Fold:  3 done!
Fold:  4 done!
Mean of all s

In [15]:
def load_score(timeout):
    with open(f'../Checkpoints/ET/ET_unsw_zeek_{timeout}.json', 'r') as f:
        loaded_results = json.load(f)
    return loaded_results


In [24]:

best_f1 = 0
worst_f1 = 1



for timeout in timeouts:
    loaded_results = load_score(timeout)
    print(loaded_results)
        
        
    if loaded_results['Mean of all scores']['f1Mean'] > best_f1: 
        best_timeout = loaded_results['Timeout']
        best_mean = loaded_results['Mean of all scores']
        best_std = loaded_results['Std of all Scores']
        best_f1 = loaded_results['Mean of all scores']['f1Mean'] 
    
    if loaded_results['Mean of all scores']['f1Mean'] <= worst_f1: 
        
        worst_timeout = loaded_results['Timeout']
        worst_mean = loaded_results['Mean of all scores']
        worst_std = loaded_results['Std of all Scores']
        worst_f1 = loaded_results['Mean of all scores']['f1Mean'] 
               
    

{'Timeout': 'default', 'Mean of all scores': {'f1Mean': 0.47086637851263075, 'accMean': 0.976235443291746, 'recMean': 0.44868973789856287, 'precMean': 0.6210600313174524}, 'Std of all Scores': {'f1Std': 0.10216837426113554, 'accStd': 0.005724921829971046, 'recStd': 0.1260704332958703, 'precStd': 0.10897177116153353}}
{'Timeout': 0.5, 'Mean of all scores': {'f1Mean': 0.4729493512626144, 'accMean': 0.9763000570866381, 'recMean': 0.45025106795409264, 'precMean': 0.6314211543706032}, 'Std of all Scores': {'f1Std': 0.10415883436271824, 'accStd': 0.005721070229415257, 'recStd': 0.1263919442193845, 'precStd': 0.11022047534371308}}


In [29]:
print("Best Timeout Combination: ", best_timeout)
print("Mean Scores (Best): ", best_mean)
print('Std Scores (Best):', best_std)

Best Timeout Combination:  default
Mean Scores (Best):  {'f1Mean': 0.43021937379884506, 'accMean': 0.976267093586246, 'recMean': 0.40952987461691004, 'precMean': 0.5597863118334097}
Std Scores (Best): {'f1Std': 0.09988257723838624, 'accStd': 0.005716066239009134, 'recStd': 0.12048625261275438, 'precStd': 0.08796330117133426}


In [30]:
print("worst Timeout Combination: ", worst_timeout)
print("Mean Scores (Worst): ", worst_mean)
print('Std Scores (Worst):', worst_std)

worst Timeout Combination:  0.5
Mean Scores (Worst):  {'f1Mean': 0.3962000542518317, 'accMean': 0.9763312766246365, 'recMean': 0.38832241291414926, 'precMean': 0.5183943543788683}
Std Scores (Worst): {'f1Std': 0.08723040793322173, 'accStd': 0.005675118223453088, 'recStd': 0.11082268873921883, 'precStd': 0.09006772797547785}


In [31]:

results = {
    'Best score': {
        'Best Timeout': best_timeout,
        'Mean Scores (Best)': best_mean,
        'Std Scores (Best)': best_std,
    },
    
    'Worst score': {
        'Worst Timeout': worst_timeout,
        'Mean Scores (Worst)': worst_mean,
        'Std Scores (Worst)': worst_std,
    },
    
    'Difference': {
        'Accuracy': (best_mean['accMean'] - worst_mean['accMean'])*100,
        'F1 Score': (best_mean['f1Mean'] - worst_mean['f1Mean'])*100,
        'Precision': (best_mean['precMean'] - worst_mean['precMean'])*100,
        'Recall': (best_mean['recMean'] - worst_mean['recMean'])*100
    }
}



with open('../results/ET_unsw_zeek.json', 'w') as f:
    json.dump(results, f, indent=4)